# ModInt: a simple modular integer type

In [2]:
struct ModInt{n} <: Integer
    k::Int

    # Constructor definition...
    # note the left side looks like the call it defines
    ModInt{n}(k::Int) where {n} = new(mod(k,n))
end

In [3]:
a = ModInt{13}(1238279873492834)

ModInt{13}(3)

In [4]:
b = ModInt{13}(9872349827349827)

ModInt{13}(12)

In [5]:
a + b

ErrorException: promotion of types ModInt{13} and ModInt{13} failed to change any arguments

To extend standard functions we need to import them.

In [6]:
import Base: +

In [7]:
+(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k + b.k)

+ (generic function with 164 methods)

In [8]:
a + b

ModInt{13}(2)

In [9]:
import Base: *, -

*(a::ModInt{n}, b::ModInt{n}) where{n} = ModInt{n}(a.k * b.k)
-(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k - b.k)
-(a::ModInt{n}) where {n} = ModInt{n}(-a.k)

- (generic function with 177 methods)

In [10]:
a * b

ModInt{13}(10)

In [11]:
a - b

ModInt{13}(4)

In [12]:
-b

ModInt{13}(1)

In [21]:
Base.show(io::IO, a::ModInt{n}) where {n} =
    get(io, :compact, false) ?
        show(io, a.k) :
        print(io, "$(a.k) mod $n")

In [18]:
a

3 mod 13

In [19]:
b

12 mod 13

In [22]:
A = map(ModInt{13}, rand(1:100, 5, 5))

5×5 Array{ModInt{13},2}:
 12   7  2  11  7
  4   7  0   2  3
  9   6  8  10  1
 12  12  3   7  9
  9   6  9   8  8

In [23]:
A .+ A

5×5 Array{ModInt{13},2}:
 11   1  4  9  1
  8   1  0  4  6
  5  12  3  7  2
 11  11  6  1  5
  5  12  5  3  3

In [26]:
A^100

5×5 Array{ModInt{13},2}:
 3  11  3  0   1
 3   3  1  8   6
 7   4  2  1  12
 8  11  4  4   1
 5   5  0  8   4

In [27]:
2A

ErrorException: promotion of types Int64 and ModInt{13} failed to change any arguments

In [28]:
A + 1

MethodError: MethodError: no method matching +(::Array{ModInt{13},2}, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:292
  +(!Matched::Missing, ::Number) at missing.jl:93
  ...

In [29]:
Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where {n} = ModInt{n}

In [33]:
promote_type(Int, ModInt{7}, BigInt)

BigInt

In [35]:
promote_type(Complex{Int}, Rational{BigInt})

Complex{Rational{BigInt}}

In [36]:
(1 + 2im) + (big(5)//big(2)^200)

1606938044258990275541962092341162602522202993782792835301381//1606938044258990275541962092341162602522202993782792835301376 + 2//1*im

In [37]:
typeof(ans)

Complex{Rational{BigInt}}

In [38]:
Base.convert(::Type{ModInt{n}}, x::Int) where {n} = ModInt{n}(x)

In [39]:
a + 1

4 mod 13

In [40]:
1 + a

4 mod 13

In [42]:
@which 1.5 + 2

+(x::Number, y::Number) in Base at promotion.jl:313

In [46]:
A^100000000000

5×5 Array{ModInt{13},2}:
 2   4  5   7   4
 5   1  7   6  11
 3  10  0   4   8
 0  11  6  12   2
 7   7  8  11   4

In [47]:
2A^100 .- 1

5×5 Array{ModInt{13},2}:
 5  8   5  12   1
 5  5   1   2  11
 0  7   3   1  10
 2  8   7   7   1
 9  9  12   2   7

### Summary

Here is all the code that defines the `ModInt` type:
```jl
struct ModInt{n} <: Integer
    k::Int

    ModInt{n}(k::Int) where {n} = new(mod(k,n))
end

import Base: +, *, -

+(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k + b.k)
*(a::ModInt{n}, b::ModInt{n}) where{n} = ModInt{n}(a.k * b.k)
-(a::ModInt{n}, b::ModInt{n}) where {n} = ModInt{n}(a.k - b.k)
-(a::ModInt{n}) where {n} = ModInt{n}(-a.k)

Base.show(io::IO, a::ModInt{n}) where {n} =
    get(io, :compact, false) ? show(io, a.k) : print(io, "$(a.k) mod $n")

Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where {n} = ModInt{n}
Base.convert(::Type{ModInt{n}}, x::Int) where {n} = ModInt{n}(x)
```

### Exercise

Add two methods that allows operations between modular integers with different modulus using the rule that they should combine in the modulus that is the `lcm` (least common multiple) of the moduli of the arguments.

**Hint:** try something, see what fails, define something to make that work.

In [48]:
x = ModInt{12}(9)

9 mod 12

In [49]:
y = ModInt{15}(13)

13 mod 15

In [51]:
lcm(12, 15)

60

In [66]:
+(a::ModInt{m}, b::ModInt{n}) where {m,n} = ModInt{lcm(m,n)}(a.k + b.k)
*(a::ModInt{m}, b::ModInt{n}) where {m,n} = ModInt{lcm(m,n)}(a.k * b.k)

* (generic function with 346 methods)

In [58]:
Base.promote_rule(::Type{ModInt{m}}, ::Type{ModInt{n}}) where {m,n} =
    ModInt{lcm(m,n)}
Base.convert(::Type{ModInt{n}}, x::ModInt) where {n} = ModInt{n}(x.k)

In [59]:
@assert x + y == ModInt{60}(22)
@assert x * y == ModInt{60}(57)

In [61]:
@assert x - y == ModInt{60}(56)

In [69]:
@which x + y

+(a::ModInt{m}, b::ModInt{n}) where {m, n} in Main at In[66]:1

In [70]:
@which x + x

+(a::ModInt{n}, b::ModInt{n}) where n in Main at In[7]:1

In [71]:
@which x - y

-(a::Integer, b::Integer) in Base at int.jl:800

In [64]:
Base.promote_typeof(x, y)

ModInt{60}

In [65]:
promote_type(ModInt{12}, ModInt{15})

ModInt{60}

In [ ]:
f(a::Number, b::Number) = f(promote(a, b)...)

In [73]:
@which 1 + 2.5

+(x::Number, y::Number) in Base at promotion.jl:313

In [74]:
@code_llvm 1 + 2.5


; Function +
; Location: promotion.jl:313
define double @"julia_+_127951684"(i64, double) {
top:
; Function promote; {
; Location: promotion.jl:284
; Function _promote; {
; Location: promotion.jl:261
; Function convert; {
; Location: number.jl:7
; Function Type; {
; Location: float.jl:60
  %2 = sitofp i64 %0 to double
;}}}}
; Function +; {
; Location: float.jl:395
  %3 = fadd double %2, %1
;}
  ret double %3
}


In [75]:
@code_native 1 + 2.5

	.section	__TEXT,__text,regular,pure_instructions
; Function + {
; Location: promotion.jl:313
; Function promote; {
; Location: promotion.jl:284
; Function _promote; {
; Location: promotion.jl:261
; Function convert; {
; Location: number.jl:7
; Function Type; {
; Location: promotion.jl:313
	vcvtsi2sdq	%rdi, %xmm1, %xmm1
;}}}}
; Function +; {
; Location: float.jl:395
	vaddsd	%xmm0, %xmm1, %xmm0
;}
	retq
	nopw	(%rax,%rax)
;}
